In [5]:
import sys
from pathlib import Path

from dotenv import load_dotenv
load_dotenv()

True

In [6]:
# LangChain 관련 라이브러리
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_huggingface import HuggingFaceEmbeddings

In [7]:
# RAGAS 관련 라이브러리
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.testset import TestsetGenerator
from ragas import EvaluationDataset, evaluate
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness

# 벡터 저장소 생성/저장

In [4]:
embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

# 기존 벡터 저장소 로드
vector_store = Chroma(
    embedding_function=embedding_model,
    persist_directory="./utils/chroma_db",
)

# 결과 확인
print(f"저장된 Document 개수: {len(vector_store.get()['ids'])}")

저장된 Document 개수: 249


In [6]:
from ragas.testset.persona import Persona

# 페르소나 정의
personas = [
    Persona(
        name="backend_team",
        role_description="CTO의 문서에 접근가능한 평사원. 한국어 사용자",
    ),
    Persona(
        name="frontend_team",
        role_description="프론트앤드 팀의 문서에 접근가능한 평사원. 한국어 사용자",
    ),
    Persona(
        name="data_ai_team",
        role_description="AI 팀의 문서에 접근가능한 평사원. 한국어 사용자",
    ),
]

In [7]:
# LLM과 임베딩 모델 초기화
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model="text-embedding-3-small"))

C:\Users\playdata\AppData\Local\Temp\ipykernel_25104\2522793028.py:3: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model="text-embedding-3-small"))


In [8]:
# TestsetGenerator 생성
generator = TestsetGenerator(
    llm=generator_llm,
    embedding_model=generator_embeddings,
    persona_list=personas
)

In [9]:
# Load and Adapt Queries
# Here we load the required query types and adapt them to the target language.
from ragas.testset.synthesizers.single_hop.specific import (
    SingleHopSpecificQuerySynthesizer,
)

distribution = [
    (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 1.0),
]

for query, _ in distribution:
    prompts = await query.adapt_prompts("korean", llm=generator_llm)
    query.set_prompts(**prompts)

In [14]:
from ragas.testset.transforms.extractors.llm_based import NERExtractor
from ragas.testset.transforms.splitters import HeadlineSplitter

transforms = [NERExtractor()]

In [15]:
from langchain.schema import Document

# Chroma에서 문서와 메타데이터 불러오기
raw = vector_store.get(include=["documents", "metadatas"])

# Document 리스트로 변환
docs = [
    Document(page_content=doc, metadata=meta)
    for doc, meta in zip(raw["documents"], raw["metadatas"])
]

In [16]:
# generate
dataset = generator.generate_with_langchain_docs(
    docs[:20],
    testset_size=20,
    transforms=transforms,
    query_distribution=distribution,
)

Applying NERExtractor:   0%|          | 0/20 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/1 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [17]:
# 데이터셋을 DataFrame으로 변환
import pandas as pd

# 컬럼너비 제한 없음
pd.set_option('display.max_colwidth', None)


dataset_df = dataset.to_pandas()
display(dataset_df)

,user_input,reference_contexts,reference,synthesizer_name
0,크랙(Crack) 앱의 주요 기능은 무엇인가요?,"[<!-- 회사: 코드노바 | 대상: 사원(백엔드) | 작성일: 2025-08-29 -->\n# 서비스 아키텍처 문서\n분류: backend | 회사: 코드노바 | 버전: v1.0 | 작성일: 2025-08-29\n\n## 1. 개요\n코드노바의 서비스 아키텍처는 생성형 AI 글쓰기·이미지·요약 플랫폼, AI 페르소나 챗봇 앱인 크랙(Crack), 그리고 대화형 광고 제작·보상 플랫폼인 Wrtn Ads를 지원하도록 설계되었습니다. 이 문서는 백엔드 시스템의 구성 요소와 상호작용을 설명합니다.\n\n## 2. 아키텍처 구성 요소\n\n### 2.1. API 서버\n- **역할**: 클라이언트와의 통신을 담당하며, 요청을 처리하고 적절한 응답을 반환합니다.\n- **기술 스택**: Node.js, Express.js\n- **검증 포인트**:\n - API 엔드포인트가 올바르게 작동하는지 확인\n - 요청 처리 속도 및 오류율 모니터링]","크랙(Crack)은 AI 페르소나 챗봇 앱으로, 코드노바의 서비스 아키텍처에서 생성형 AI 글쓰기·이미지·요약 플랫폼과 함께 지원됩니다.",single_hop_specific_query_synthesizer
1,RabbitMQ의 역할은 무엇인가요?,"[### 2.2. 데이터베이스\n- **역할**: 사용자 데이터, 콘텐츠, 로그 등을 저장합니다.\n- **기술 스택**: PostgreSQL\n- **검증 포인트**:\n - 데이터베이스 연결 상태 확인\n - 쿼리 성능 분석 및 최적화\n\n### 2.3. 캐시 서버\n- **역할**: 데이터베이스 부하를 줄이고 응답 속도를 높이기 위해 자주 조회되는 데이터를 캐싱합니다.\n- **기술 스택**: Redis\n- **검증 포인트**:\n - 캐시 적중률 모니터링\n - 캐시 데이터의 일관성 확인\n\n### 2.4. 메시지 큐\n- **역할**: 비동기 작업 처리를 위해 시스템 간 메시지를 전달합니다.\n- **기술 스택**: RabbitMQ\n- **검증 포인트**:\n - 메시지 전송 성공률 확인\n - 큐의 길이 및 처리 속도 모니터링]",RabbitMQ는 비동기 작업 처리를 위해 시스템 간 메시지를 전달하는 메시지 큐입니다.,single_hop_specific_query_synthesizer
2,프론트엔드 팀에서 JWT의 역할은 무엇인가요?,[### 2.5. 파일 저장소\n- **역할**: 이미지 및 기타 미디어 파일을 저장합니다.\n- **기술 스택**: AWS S3\n- **검증 포인트**:\n - 파일 업로드 및 다운로드 기능 확인\n - 저장소 사용량 모니터링\n\n## 3. 서비스 흐름\n\n1. **사용자 요청**: 클라이언트에서 API 서버에 요청을 보냅니다.\n2. **API 처리**: API 서버는 요청을 처리하고 필요한 경우 데이터베이스 또는 캐시 서버에 접근합니다.\n3. **비동기 작업**: 일부 요청은 메시지 큐에 전송되어 비동기로 처리됩니다.\n4. **응답 반환**: API 서버는 클라이언트에 응답을 반환합니다.\n\n## 4. 보안\n- **인증**: JWT(JSON Web Token)를 사용하여 사용자 인증을 처리합니다.\n- **데이터 암호화**: 전송 중 데이터 암호화를 위해 HTTPS를 사용합니다.\n- **검증 포인트**:\n - 인증 토큰의 유효성 검사\n - 보안 로그 모니터링],JWT(JSON Web Token)는 사용자 인증을 처리하는 데 사용됩니다.,single_hop_specific_query_synthesizer
3,"코드노바의 백엔드 서비스 아키텍처는 어떻게 설계되었고, 그 목적은 무엇인가요?","[## 5. 모니터링 및 로깅\n- **모니터링 도구**: Prometheus 및 Grafana를 사용하여 시스템 성능 및 상태를 모니터링합니다.\n- **로깅**: ELK 스택(Elasticsearch, Logstash, Kibana)을 사용하여 로그를 수집하고 분석합니다.\n- **검증 포인트**:\n - 주요 메트릭스 확인\n - 시스템 오류 및 경고 모니터링\n\n## 6. 결론\n코드노바의 백엔드 서비스 아키텍처는 확장 가능하고 안정적인 서비스를 제공하기 위해 설계되었습니다. 각 구성 요소는 독립적으로 운영되며, 서로 긴밀하게 연결되어 있습니다. 지속적인 모니터링과 최적화를 통해 서비스의 품질을 유지하고 향상시킬 수 있습니다.\n\n---\n\n다음 개정 제안: 서비스 아키텍처의 각 구성 요소에 대한 세부적인 기술 스택과 버전 정보를 추가하는 것이 좋습니다. 또한, 보안 관련 사항을 더 구체적으로 설명할 필요가 있습니다.]","코드노바의 백엔드 서비스 아키텍처는 확장 가능하고 안정적인 서비스를 제공하기 위해 설계되었습니다. 각 구성 요소는 독립적으로 운영되며, 서로 긴밀하게 연결되어 있습니다. 지속적인 모니터링과 최적화를 통해 서비스의 품질을 유지하고 향상시킬 수 있습니다.",single_hop_specific_query_synthesizer
4,MFA는 뭐하는거야?,[<!-- 회사: 코드노바 | 대상: 사원(백엔드) | 작성일: 2025-08-29 -->\n# 보안/인증 가이드\n분류: backend | 회사: 코드노바 | 버전: v1.0 | 작성일: 2025-08-29\n\n## 1. 보안 기본 원칙\n\n### 1.1 최소 권한 원칙\n- 사용자는 자신의 업무에 필요한 최소한의 권한만 부여받아야 합니다.\n- 정기적으로 권한을 검토하고 불필요한 권한은 즉시 회수합니다.\n\n### 1.2 데이터 암호화\n- 모든 민감 데이터는 저장 시 암호화하여 보호합니다.\n- 전송되는 데이터는 SSL/TLS 프로토콜을 사용하여 암호화합니다.\n\n### 1.3 정기적인 보안 점검\n- 보안 점검을 정기적으로 실시하여 취약점을 발견하고 수정합니다.\n- 외부 보안 전문가에 의한 감사도 고려합니다.\n\n## 2. 사용자 인증\n\n### 2.1 인증 방법\n- 기본 인증 방식으로는 이메일과 비밀번호 조합을 사용합니다.\n- 추가적인 보안 강화를 위해 다중 인증(MFA)을 도입합니다.],MFA는 추가적인 보안 강화를 위해 도입되는 다중 인증 방법입니다.,single_hop_specific_query_synthesizer
5,JWT는 API 접근 시 어떤 역할을 합니까?,"[### 2.2 비밀번호 관리\n- 비밀번호는 최소 8자 이상, 대문자, 소문자, 숫자, 특수문자를 포함해야 합니다.\n- 비밀번호는 주기적으로 변경하도록 유도합니다.\n\n### 2.3 세션 관리\n- 사용자가 로그인 후 일정 시간 동안 활동이 없으면 자동으로 로그아웃됩니다.\n- 세션 ID는 예측 불가능한 값으로 생성하고, 매 요청 시 갱신합니다.\n\n## 3. API 보안\n\n### 3.1 인증 토큰 사용\n- API 접근 시 JWT(JSON Web Token) 또는 OAuth2를 사용하여 인증합니다.\n- 토큰은 유효 기간을 설정하고, 만료된 토큰은 사용하지 못하도록 합니다.\n\n### 3.2 CORS 설정\n- Cross-Origin Resource Sharing(CORS) 정책을 설정하여 허용된 도메인에서만 API 접근을 허용합니다.\n\n### 3.3 요청 검증\n- 모든 API 요청에 대해 유효성을 검증합니다.\n- SQL 인젝션, XSS 공격 등을 방지하기 위한 필터링을 적용합니다.]","JWT(JSON Web Token)는 API 접근 시 인증을 위해 사용되며, 토큰은 유효 기간을 설정하고 만료된 토큰은 사용하지 못하도록 합니다.",single_hop_specific_query_synthesizer
6,코드노바의 백엔드 개발팀이 보안 및 인증을 관리하기 위한 기본 지침은 무엇입니까?,"[## 4. 로그 관리\n\n### 4.1 로그 기록\n- 모든 인증 시도 및 API 호출에 대한 로그를 기록합니다.\n- 로그에는 사용자 ID, IP 주소, 요청 시간, 요청 내용 등을 포함합니다.\n\n### 4.2 로그 분석\n- 정기적으로 로그를 분석하여 비정

In [8]:
import pandas as pd

dataset_df = pd.read_csv('ragas_dataset.csv')

display(dataset_df)

,user_input,reference_contexts,reference,synthesizer_name
0,엑스프레스.js는 코드노바의 API 서버에서 어떤 역할을 하나요?,['<!-- 회사: 코드노바 | 대상: 사원(백엔드) | 작성일: 2025-08-2...,"Express.js는 코드노바의 API 서버에서 클라이언트와의 통신을 담당하며, 요...",single_hop_specific_query_synthesizer
1,PostgreSQL의 주요 역할은 무엇입니까?,"['### 2.2. 데이터베이스\n- **역할**: 사용자 데이터, 콘텐츠, 로그 ...","PostgreSQL의 주요 역할은 사용자 데이터, 콘텐츠, 로그 등을 저장하는 것입니다.",single_hop_specific_query_synthesizer
2,데이터 전송 중 보안을 위해 HTTPS를 사용하는 이유는 무엇인가요?,['### 2.5. 파일 저장소\n- **역할**: 이미지 및 기타 미디어 파일을 ...,데이터 암호화를 위해 HTTPS를 사용하여 전송 중 데이터를 보호합니다.,single_hop_specific_query_synthesizer
3,다중 인증(MFA)이란 무엇인가요?,['<!-- 회사: 코드노바 | 대상: 사원(백엔드) | 작성일: 2025-08-2...,다중 인증(MFA)은 기본 인증 방식인 이메일과 비밀번호 조합에 추가적인 보안 강화...,single_hop_specific_query_synthesizer
4,XSS 공격을 방지하기 위한 방법은 무엇인가요?,"['### 2.2 비밀번호 관리\n- 비밀번호는 최소 8자 이상, 대문자, 소문자,...","모든 API 요청에 대해 유효성을 검증하고, XSS 공격 등을 방지하기 위한 필터링...",single_hop_specific_query_synthesizer
5,백엔드 개발팀이 로그 관리와 사고 대응을 효과적으로 수행하기 위해 어떤 절차를 따르...,['## 4. 로그 관리\n\n### 4.1 로그 기록\n- 모든 인증 시도 및 A...,백엔드 개발팀은 로그 관리에서 모든 인증 시도 및 API 호출에 대한 로그를 기록하...,single_hop_specific_query_synthesizer
6,v1.0 버전 에러 핸들링 매뉴얼에서 에러 종류 뭐 있나요?,['<!-- 회사: 코드노바 | 대상: 사원(백엔드) | 작성일: 2025-08-2...,"에러는 크게 두 가지로 분류됩니다. 첫째, 클라이언트 에러로 사용자의 요청에 문제가...",single_hop_specific_query_synthesizer
7,코드노바 백엔드팀의 배포 및 운영 과정에서 코드 검토와 테스트 수행은 어떻게 이루어...,['<!-- 회사: 코드노바 | 대상: 사원(백엔드) | 작성일: 2025-08-2...,코드노바 백엔드팀의 배포 및 운영 과정에서 코드 검토는 모든 변경 사항에 대해 코드...,single_hop_specific_query_synthesizer
8,백엔드팀이 배포 및 운영을 효율적으로 수행하기 위해 따라야 할 체크리스트는 무엇인가요?,['## 체크리스트\n\n- [ ] 코드 리뷰 완료\n- [ ] 모든 테스트 통과\...,백엔드팀이 배포 및 운영을 효율적으로 수행하기 위해 따라야 할 체크리스트는 다음과 ...,single_hop_specific_query_synthesizer
9,데이터베이스 스키마에서 사용자 엔티티는 어떤 정보를 저장하나요?,['<!-- 회사: 코드노바 | 대상: 사원(백엔드) | 작성일: 2025-08-2...,사용자(User) 엔티티는 사용자 정보를 저장합니다.,single_hop_specific_query_synthesizer


## 02.RAG 체인 구성

검색기와 생성 모델을 결합한 RAG 체인을 구성한다.

In [1]:
from dotenv import load_dotenv

load_dotenv()

from pathlib import Path
import sys

root = Path.cwd()
while not (root / "utils4").exists() and root != root.parent:
    root = root.parent
sys.path.insert(0, str(root))

from utils4.rag import chat

response = chat({"history": [
    {"role": "user", "content": "코드노바의 API 서버 기술스택 알려줘"}], "tone":"formal", "permission":"cto"})

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-m3
INFO:utils4.service:-------- Chat Request - Permission: cto, Tone: formal
INFO:utils4.service:-------- Tools Prompt: 사용자는 cto로서, 모든 팀의 문서를 열람할 수 있는 개발팀 최고 관리자입니다.
                당신은 <tools></tools> 안에 있는 tool을 호출하여 문서를 검색할 수 있습니다.
                일상적인 질문(ex: 안녕, 안녕하세요, 반가워 등)의 경우, tool 호출 없이 바로 답변하세요.

                # Tools

                You may call one or more functions to assist with the user query.

                You are provided with function signatures within <tools></tools> XML tags:
                <tools>
                {"type": "function", "function": {"name": "cto_search", "description": "사내 문서 검색을 위한 도구입니다. 대화 내역을 바탕으로 사용자가 원하는 문서를 찾고, 관련된 문서를 반환합니다.", "parameters": {"type": "object", "properties": {"keyword": {"type": "string", "description": "검색할 문서 키워드 (예: '코드노바 API 서버 설정')"}}, "required": ["k

In [13]:
response = chat({"history": [
    {"role": "user", "content": "코드노바의 API 서버 기술스택 알려줘"}], "tone":"formal", "permission":"cto"})

INFO:utils4.service:-------- Chat Request - Permission: cto, Tone: formal
INFO:utils4.service:-------- Tools Prompt: 사용자는 cto로서, 모든 팀의 문서를 열람할 수 있는 개발팀 최고 관리자입니다.
                당신은 <tools></tools> 안에 있는 tool을 호출하여 문서를 검색할 수 있습니다.
                일상적인 질문(ex: 안녕, 안녕하세요, 반가워 등)의 경우, tool 호출 없이 바로 답변하세요.

                # Tools

                You may call one or more functions to assist with the user query.

                You are provided with function signatures within <tools></tools> XML tags:
                <tools>
                {"type": "function", "function": {"name": "cto_search", "description": "사내 문서 검색을 위한 도구입니다. 대화 내역을 바탕으로 사용자가 원하는 문서를 찾고, 관련된 문서를 반환합니다.", "parameters": {"type": "object", "properties": {"keyword": {"type": "string", "description": "검색할 문서 키워드 (예: '코드노바 API 서버 설정')"}}, "required": ["keyword"], "additionalProperties": false}}}
                </tools>

                For each function call, return a json object with function name and arguments within <t

In [14]:
print(response['response'])

코드노바의 API 서버 기술스택은 Node.js와 Express.js로 구성되어 있습니다.


In [15]:
print(response['result'])
type(response['result'])

['검색 결과:\n-----\n<!-- 회사: 코드노바 | 대상: 사원(백엔드) | 작성일: 2025-08-29 -->\n# 서비스 아키텍처 문서\n분류: backend | 회사: 코드노바 | 버전: v1.0 | 작성일: 2025-08-29\n\n## 1. 개요\n코드노바의 서비스 아키텍처는 생성형 AI 글쓰기·이미지·요약 플랫폼, AI 페르소나 챗봇 앱인 크랙(Crack), 그리고 대화형 광고 제작·보상 플랫폼인 Wrtn Ads를 지원하도록 설계되었습니다. 이 문서는 백엔드 시스템의 구성 요소와 상호작용을 설명합니다.\n\n## 2. 아키텍처 구성 요소\n\n### 2.1. API 서버\n- **역할**: 클라이언트와의 통신을 담당하며, 요청을 처리하고 적절한 응답을 반환합니다.\n- **기술 스택**: Node.js, Express.js\n- **검증 포인트**:\n  - API 엔드포인트가 올바르게 작동하는지 확인\n  - 요청 처리 속도 및 오류율 모니터링 [[ref1]]\n<!-- 회사: 코드노바 | 대상: 사원(프론트엔드) | 작성일: 2025-08-29 -->\n# 프론트엔드 아키텍처 문서\n분류: frontend | 회사: 코드노바 | 버전: v1.0 | 작성일: 2025-08-29\n\n---\n\n## 1. 개요\n코드노바의 프론트엔드 아키텍처는 사용자 경험을 극대화하고, 유지보수성을 높이며, 확장성을 고려하여 설계되었습니다. 본 문서는 프론트엔드 개발자가 이해하고 실행할 수 있도록 아키텍처의 구성 요소와 원칙을 설명합니다.\n\n## 2. 아키텍처 구성 요소\n\n### 2.1. 기술 스택\n- **프레임워크**: React, Vue.js 또는 Angular 중 하나를 선택하여 사용\n- **상태 관리**: Redux, Vuex 또는 Context API를 사용하여 애플리케이션 상태 관리\n- **스타일링**: CSS Modules, Styled-components 또는 SCSS 사용\n- **빌드 도구**: Webpack, Babel 등 

list

## 03.RAGAS기반 평가

RAGAS는 RAG(Retrieval-Augmented Generation) 시스템을 평가하는 자동화된 참조 없는(reference-free) 평가 프레임워크입니다. 즉, 사람이 만든 정답 데이터(ground truth)가 없이도 RAG 시스템의 검색과 생성 단계를 자동으로 평가할 수 있도록 설계된 도구입니다. RAGAS는 LLM을 활용해 평가를 수행하며, 검색된 컨텍스트와 생성된 답변의 충실도, 관련성, 정밀도, 재현율 등 다양한 측면을 측정합니다.

### RAGAS 주요 평가 지표
- **Faithfulness(충실도)**: 생성된 답변이 주어진 컨텍스트 정보에 얼마나 충실한지를 평가합니다. 답변 내용이 컨텍스트에서 실제로 뒷받침되는지 보는 지표입니다.
- **Answer Relevancy(답변 관련성)**: 답변이 원 질문과 얼마나 관련성이 높은지를 측정합니다.
- **Context Precision(컨텍스트 정밀도)**: 검색된 컨텍스트 문서가 질문에 적절한 정보인지, 관련된 문서가 상위에 있는지를 평가합니다.
- **Context Recall(컨텍스트 재현율)**: 답변을 생성하는 데 필요한 컨텍스트를 얼마나 잘 검색했는지 평가합니다.


In [9]:
# 평가용 데이터 로드
import pandas as pd

# 같은 폴더(현재 작업 디렉토리)에 저장했다면
dataset_df = pd.read_csv("./ragas_dataset.csv")
print(dataset_df.shape)

eval_dataset = dataset_df[['user_input', 'reference_contexts', 'reference']]
display(eval_dataset)

(26, 4)


,user_input,reference_contexts,reference
0,엑스프레스.js는 코드노바의 API 서버에서 어떤 역할을 하나요?,['<!-- 회사: 코드노바 | 대상: 사원(백엔드) | 작성일: 2025-08-2...,"Express.js는 코드노바의 API 서버에서 클라이언트와의 통신을 담당하며, 요..."
1,PostgreSQL의 주요 역할은 무엇입니까?,"['### 2.2. 데이터베이스\n- **역할**: 사용자 데이터, 콘텐츠, 로그 ...","PostgreSQL의 주요 역할은 사용자 데이터, 콘텐츠, 로그 등을 저장하는 것입니다."
2,데이터 전송 중 보안을 위해 HTTPS를 사용하는 이유는 무엇인가요?,['### 2.5. 파일 저장소\n- **역할**: 이미지 및 기타 미디어 파일을 ...,데이터 암호화를 위해 HTTPS를 사용하여 전송 중 데이터를 보호합니다.
3,다중 인증(MFA)이란 무엇인가요?,['<!-- 회사: 코드노바 | 대상: 사원(백엔드) | 작성일: 2025-08-2...,다중 인증(MFA)은 기본 인증 방식인 이메일과 비밀번호 조합에 추가적인 보안 강화...
4,XSS 공격을 방지하기 위한 방법은 무엇인가요?,"['### 2.2 비밀번호 관리\n- 비밀번호는 최소 8자 이상, 대문자, 소문자,...","모든 API 요청에 대해 유효성을 검증하고, XSS 공격 등을 방지하기 위한 필터링..."
5,백엔드 개발팀이 로그 관리와 사고 대응을 효과적으로 수행하기 위해 어떤 절차를 따르...,['## 4. 로그 관리\n\n### 4.1 로그 기록\n- 모든 인증 시도 및 A...,백엔드 개발팀은 로그 관리에서 모든 인증 시도 및 API 호출에 대한 로그를 기록하...
6,v1.0 버전 에러 핸들링 매뉴얼에서 에러 종류 뭐 있나요?,['<!-- 회사: 코드노바 | 대상: 사원(백엔드) | 작성일: 2025-08-2...,"에러는 크게 두 가지로 분류됩니다. 첫째, 클라이언트 에러로 사용자의 요청에 문제가..."
7,코드노바 백엔드팀의 배포 및 운영 과정에서 코드 검토와 테스트 수행은 어떻게 이루어...,['<!-- 회사: 코드노바 | 대상: 사원(백엔드) | 작성일: 2025-08-2...,코드노바 백엔드팀의 배포 및 운영 과정에서 코드 검토는 모든 변경 사항에 대해 코드...
8,백엔드팀이 배포 및 운영을 효율적으로 수행하기 위해 따라야 할 체크리스트는 무엇인가요?,['## 체크리스트\n\n- [ ] 코드 리뷰 완료\n- [ ] 모든 테스트 통과\...,백엔드팀이 배포 및 운영을 효율적으로 수행하기 위해 따라야 할 체크리스트는 다음과 ...
9,데이터베이스 스키마에서 사용자 엔티티는 어떤 정보를 저장하나요?,['<!-- 회사: 코드노바 | 대상: 사원(백엔드) | 작성일: 2025-08-2...,사용자(User) 엔티티는 사용자 정보를 저장합니다.


In [16]:
# 평가용 데이터셋 생성
evaluated_dataset = []

# 각 행에 대해 RAG 체인을 호출하여 결과를 저장
for row in eval_dataset.itertuples():
    query = row.user_input  # 사용자 입력
    response = chat({"history": [
    {"role": "user", "content": query}], "tone":"formal", "permission":"cto"})
    retrieved_contexts = response['result']  # 실제 검색된 문서

    reference = row.reference  # 정답
    reference_contexts = row.reference_contexts  # 정답 참조 컨텍스트


    evaluated_dataset.append(
        {
            "user_input": query,
            "retrieved_contexts": retrieved_contexts,
            "response": response['response'],
            "reference": reference,
            # "reference_contexts": reference_contexts,
        }
    )

# RAGAS 평가 데이터셋 생성
ragas_evaluated_dataset = EvaluationDataset.from_list(evaluated_dataset)

# 데이터 저장
ragas_evaluated_dataset.to_pandas().to_csv('ragas_evaluated_dataset.csv', index=False)

INFO:utils4.service:-------- Chat Request - Permission: cto, Tone: formal
INFO:utils4.service:-------- Tools Prompt: 사용자는 cto로서, 모든 팀의 문서를 열람할 수 있는 개발팀 최고 관리자입니다.
                당신은 <tools></tools> 안에 있는 tool을 호출하여 문서를 검색할 수 있습니다.
                일상적인 질문(ex: 안녕, 안녕하세요, 반가워 등)의 경우, tool 호출 없이 바로 답변하세요.

                # Tools

                You may call one or more functions to assist with the user query.

                You are provided with function signatures within <tools></tools> XML tags:
                <tools>
                {"type": "function", "function": {"name": "cto_search", "description": "사내 문서 검색을 위한 도구입니다. 대화 내역을 바탕으로 사용자가 원하는 문서를 찾고, 관련된 문서를 반환합니다.", "parameters": {"type": "object", "properties": {"keyword": {"type": "string", "description": "검색할 문서 키워드 (예: '코드노바 API 서버 설정')"}}, "required": ["keyword"], "additionalProperties": false}}}
                </tools>

                For each function call, return a json object with function name and arguments within <t

In [17]:
display(ragas_evaluated_dataset.to_pandas())

,user_input,retrieved_contexts,response,reference
0,엑스프레스.js는 코드노바의 API 서버에서 어떤 역할을 하나요?,[검색 결과:\n-----\n### 2.2. 데이터베이스\n- **역할**: 사용자...,"엑스프레스.js는 코드노바 API 서버에서 클라이언트와의 통신을 담당하며, 요청을 ...","Express.js는 코드노바의 API 서버에서 클라이언트와의 통신을 담당하며, 요..."
1,PostgreSQL의 주요 역할은 무엇입니까?,[검색 결과:\n-----\n### 2.2. 데이터베이스\n- **역할**: 사용자...,"PostgreSQL의 주요 역할은 사용자 데이터, 콘텐츠, 로그 등을 저장하는 데이...","PostgreSQL의 주요 역할은 사용자 데이터, 콘텐츠, 로그 등을 저장하는 것입니다."
2,데이터 전송 중 보안을 위해 HTTPS를 사용하는 이유는 무엇인가요?,[검색 결과:\n-----\n## 3. 데이터 보호\n- **민감한 데이터 처리**...,HTTPS를 사용하는 이유는 데이터 전송 중 보안을 강화하기 위함입니다. HTTPS...,데이터 암호화를 위해 HTTPS를 사용하여 전송 중 데이터를 보호합니다.
3,다중 인증(MFA)이란 무엇인가요?,[검색 결과:\n-----\n## 5. 리스크 한계 및 보완책\n- **리스크 한계...,"다중 인증(MFA, Multi-Factor Authentication)은 사용자가 ...",다중 인증(MFA)은 기본 인증 방식인 이메일과 비밀번호 조합에 추가적인 보안 강화...
4,XSS 공격을 방지하기 위한 방법은 무엇인가요?,[검색 결과:\n-----\n## 3. 데이터 보호\n- **민감한 데이터 처리**...,XSS 공격을 방지하기 위한 주요 방법은 다음과 같습니다.\n\n1. 사용자 입력에...,"모든 API 요청에 대해 유효성을 검증하고, XSS 공격 등을 방지하기 위한 필터링..."
5,백엔드 개발팀이 로그 관리와 사고 대응을 효과적으로 수행하기 위해 어떤 절차를 따르...,[검색 결과:\n-----\n## 4. 로그 관리\n\n### 4.1 로그 기록\n...,백엔드 개발팀은 로그 관리와 사고 대응을 다음과 같은 절차로 수행하고 있습니다.\n...,백엔드 개발팀은 로그 관리에서 모든 인증 시도 및 API 호출에 대한 로그를 기록하...
6,v1.0 버전 에러 핸들링 매뉴얼에서 에러 종류 뭐 있나요?,[검색 결과:\n-----\n<!-- 회사: 코드노바 | 대상: 사원(백엔드) | ...,"v1.0 버전 에러 핸들링 매뉴얼에 따르면, 에러 종류는 크게 두 가지로 분류됩니다...","에러는 크게 두 가지로 분류됩니다. 첫째, 클라이언트 에러로 사용자의 요청에 문제가..."
7,코드노바 백엔드팀의 배포 및 운영 과정에서 코드 검토와 테스트 수행은 어떻게 이루어...,[검색 결과:\n-----\n<!-- 회사: 코드노바 | 대상: 사원(백엔드) | ...,코드노바 백엔드팀의 배포 및 운영 과정에서 코드 검토와 테스트는 다음과 같이 이루어...,코드노바 백엔드팀의 배포 및 운영 과정에서 코드 검토는 모든 변경 사항에 대해 코드...
8,백엔드팀이 배포 및 운영을 효율적으로 수행하기 위해 따라야 할 체크리스트는 무엇인가요?,[검색 결과:\n-----\n## 체크리스트\n\n- [ ] 코드 리뷰 완료\n- ...,백엔드팀이 배포 및 운영을 효율적으로 수행하기 위해 따라야 할 체크리스트는 다음과 ...,백엔드팀이 배포 및 운영을 효율적으로 수행하기 위해 따라야 할 체크리스트는 다음과 ...
9,데이터베이스 스키마에서 사용자 엔티티는 어떤 정보를 저장하나요?,[검색 결과:\n-----\n<!-- 회사: 코드노바 | 대상: 사원(백엔드) | ...,사용자 엔티티(User)는 사용자 정보를 저장하는 데이터베이스 스키마의 주요 엔티티...,사용자(User) 엔티티는 사용자 정보를 저장합니다.


### RAGAS 평가 실행

**ragas.metrics 주요 클래스 설명**

- **LLMContextRecall**  
  
  주어진 답변이 원문(문맥)에서 제시된 정보를 얼마나 잘 회상(recall)했는지 평가하는 클래스이다.
  
  예를 들어, 원문에 중요한 사실 10개가 있는데 답변이 그중 몇 개를 잘 포함했는지 측정한다.
  즉, 잊지 않고 잘 회상했는지 보는 지표다.

- **Faithfulness**  
  답변이 원본(문맥) 정보에 기반하여 얼마나 충실하고 일관되게 생성되었는지 평가하는 클래스이다.

  답변 내 내용이 문맥과 어긋나지 않고, 문맥에서 유추 가능한 사실들로만 이루어졌는지를 본다.

  낮은 faithfulness 값은 모델이 문맥에 없는 정보를 만들어내거나 왜곡하여 답변함을 의미할 수 있어, 환각 발생 정도를 간접적으로 보여준다.

- **FactualCorrectness**  
  답변의 내용이 실제 사실과 얼마나 정확하게 일치하는지를 평가하는 클래스이다.

  문맥뿐 아니라 절대적인 사실관계(정확성)와 맞는지 따진다.

  즉, 문맥이 아니라 객관적인 사실에 맞는지를 측정하는 지표다.

In [18]:
# LLM 래퍼 생성
evaluator_llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)
evaluator_llm = LangchainLLMWrapper(evaluator_llm)

# 평가 메트릭 정의
metrics = [
    LLMContextRecall(),    # 검색된 컨텍스트의 회수율
    Faithfulness(),        # 생성된 답변의 충실도
    FactualCorrectness()   # 사실적 정확성
]

In [19]:
# 평가 실행
result = evaluate(
    dataset=ragas_evaluated_dataset,  # 평가 데이터셋
    metrics=metrics,             # 평가 메트릭
    llm=evaluator_llm,          # LLM 래퍼
)

print(result)

Evaluating:   0%|          | 0/78 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

{'context_recall': 0.8462, 'faithfulness': 0.6569, 'factual_correctness(mode=f1)': 0.5250}


In [20]:
# 결과를 DataFrame으로 변환
result_df = result.to_pandas()

# 결과 저장
result_df.to_csv('ragas_evaluation_result.csv', index=False)

display(result_df)

,user_input,retrieved_contexts,response,reference,context_recall,faithfulness,factual_correctness(mode=f1)
0,엑스프레스.js는 코드노바의 API 서버에서 어떤 역할을 하나요?,[검색 결과:\n-----\n### 2.2. 데이터베이스\n- **역할**: 사용자...,"엑스프레스.js는 코드노바 API 서버에서 클라이언트와의 통신을 담당하며, 요청을 ...","Express.js는 코드노바의 API 서버에서 클라이언트와의 통신을 담당하며, 요...",1.0,0.625000,0.67
1,PostgreSQL의 주요 역할은 무엇입니까?,[검색 결과:\n-----\n### 2.2. 데이터베이스\n- **역할**: 사용자...,"PostgreSQL의 주요 역할은 사용자 데이터, 콘텐츠, 로그 등을 저장하는 데이...","PostgreSQL의 주요 역할은 사용자 데이터, 콘텐츠, 로그 등을 저장하는 것입니다.",1.0,0.400000,0.50
2,데이터 전송 중 보안을 위해 HTTPS를 사용하는 이유는 무엇인가요?,[검색 결과:\n-----\n## 3. 데이터 보호\n- **민감한 데이터 처리**...,HTTPS를 사용하는 이유는 데이터 전송 중 보안을 강화하기 위함입니다. HTTPS...,데이터 암호화를 위해 HTTPS를 사용하여 전송 중 데이터를 보호합니다.,1.0,0.750000,0.67
3,다중 인증(MFA)이란 무엇인가요?,[검색 결과:\n-----\n## 5. 리스크 한계 및 보완책\n- **리스크 한계...,"다중 인증(MFA, Multi-Factor Authentication)은 사용자가 ...",다중 인증(MFA)은 기본 인증 방식인 이메일과 비밀번호 조합에 추가적인 보안 강화...,1.0,0.600000,0.57
4,XSS 공격을 방지하기 위한 방법은 무엇인가요?,[검색 결과:\n-----\n## 3. 데이터 보호\n- **민감한 데이터 처리**...,XSS 공격을 방지하기 위한 주요 방법은 다음과 같습니다.\n\n1. 사용자 입력에...,"모든 API 요청에 대해 유효성을 검증하고, XSS 공격 등을 방지하기 위한 필터링...",1.0,0.833333,0.33
5,백엔드 개발팀이 로그 관리와 사고 대응을 효과적으로 수행하기 위해 어떤 절차를 따르...,[검색 결과:\n-----\n## 4. 로그 관리\n\n### 4.1 로그 기록\n...,백엔드 개발팀은 로그 관리와 사고 대응을 다음과 같은 절차로 수행하고 있습니다.\n...,백엔드 개발팀은 로그 관리에서 모든 인증 시도 및 API 호출에 대한 로그를 기록하...,1.0,1.000000,0.94
6,v1.0 버전 에러 핸들링 매뉴얼에서 에러 종류 뭐 있나요?,[검색 결과:\n-----\n<!-- 회사: 코드노바 | 대상: 사원(백엔드) | ...,"v1.0 버전 에러 핸들링 매뉴얼에 따르면, 에러 종류는 크게 두 가지로 분류됩니다...","에러는 크게 두 가지로 분류됩니다. 첫째, 클라이언트 에러로 사용자의 요청에 문제가...",1.0,1.000000,0.86
7,코드노바 백엔드팀의 배포 및 운영 과정에서 코드 검토와 테스트 수행은 어떻게 이루어...,[검색 결과:\n-----\n<!-- 회사: 코드노바 | 대상: 사원(백엔드) | ...,코드노바 백엔드팀의 배포 및 운영 과정에서 코드 검토와 테스트는 다음과 같이 이루어...,코드노바 백엔드팀의 배포 및 운영 과정에서 코드 검토는 모든 변경 사항에 대해 코드...,1.0,1.000000,0.64
8,백엔드팀이 배포 및 운영을 효율적으로 수행하기 위해 따라야 할 체크리스트는 무엇인가요?,[검색 결과:\n-----\n## 체크리스트\n\n- [ ] 코드 리뷰 완료\n- ...,백엔드팀이 배포 및 운영을 효율적으로 수행하기 위해 따라야 할 체크리스트는 다음과 ...,백엔드팀이 배포 및 운영을 효율적으로 수행하기 위해 따라야 할 체크리스트는 다음과 ...,1.0,1.000000,0.82
9,데이터베이스 스키마에서 사용자 엔티티는 어떤 정보를 저장하나요?,[검색 결과:\n-----\n<!-- 회사: 코드노바 | 대상: 사원(백엔드) | ...,사용자 엔티티(User)는 사용자 정보를 저장하는 데이터베이스 스키마의 주요 엔티티...,사용자(User) 엔티티는 사용자 정보를 저장합니다.,1.0,0.400000,0.40


### 평가지표 해석 및 개선방안

1. context_recall (문맥 리콜, 0~1):

    검색된 문서나 문맥에 답변 작성을 위한 필요한 정보가 얼마나 포함되어 있는지를 나타냅니다. 값이 0.5라면, 필요한 정보가 절반 정도는 검색되어 있다는 뜻으로, 문맥의 충분성이 보통 수준임을 뜻합니다.

2. faithfulness (충실도, 0~1):

    생성된 답변이 검색된 문맥 내 정보에 얼마나 사실적으로 충실한지를 평가합니다. 0.5 정도면 절반 정도 문맥 정보를 충실히 반영했으나, 환각(허위 정보) 발생 가능성이 중간 정도임을 의미합니다.

3. factual_correctness (사실 정확성, F1 점수, 0~1)

    생성 답변의 사실적 정확도를 정밀도와 재현율의 조합으로 평가하는 지표입니다. 0.17은 비교적 낮은 점수로, 생성 답변에 사실과 다른 정보가 많이 포함되었을 가능성이 높음을 뜻합니다. 즉, 환각 현상이 꽤 존재할 수 있습니다.


**개선 방안**

- **검색 성능 향상**: 더 정확한 임베딩 모델이나 하이브리드 검색 방법을 고려한다
- **프롬프트 최적화**: 더 구체적이고 명확한 지시사항을 포함한 프롬프트를 설계한다
- **데이터 품질 개선**: 더 정확하고 완전한 참조 데이터를 확보한다
- **후처리 단계 추가**: 사실 검증 단계를 추가하여 정확성을 향상시킨다
- **다양한 검색 전략**(하이브리드 검색, 리랭킹 등) 실험
- **커스텀 평가 메트릭 개발**
- **실시간 평가 시스템 구축**
- **A/B 테스트를 통한 시스템 최적화**